In [1]:
(in-package "ACL2")

 "ACL2"


In [2]:
(include-book "tools/include-raw" :dir :system)
(include-book "acl2s/aspf/interface/top" :dir :system 
              :ttags ((:acl2s-interface)))
(include-book "std/strings/top" :dir :system)


Summary
Form:  ( INCLUDE-BOOK "tools/include-raw" ...)
Rules: NIL
Time:  0.06 seconds (prove: 0.00, print: 0.00, other: 0.06)
 "/home/acl2/books/tools/include-raw.lisp"

TTAG NOTE (for included book): Adding ttag :ACL2S-INTERFACE from book /home/acl2/books/acl2s/aspf/interface/top.

Summary
Form:  ( INCLUDE-BOOK "acl2s/aspf/interface/top" ...)
Rules: NIL
Time:  0.05 seconds (prove: 0.00, print: 0.00, other: 0.05)
 "/home/acl2/books/acl2s/aspf/interface/top.lisp"

Summary
Form:  ( INCLUDE-BOOK "std/strings/top" ...)
Rules: NIL
Time:  0.62 seconds (prove: 0.00, print: 0.00, other: 0.61)
 "/home/acl2/books/std/strings/top.lisp"


In [3]:
;; List of forbidden function/macro symbols that could break safety
(defconst *forbidden-symbols*
  '(;; System calls and raw Lisp access
    sys-call sys-call-status sys-call+
    set-raw-mode set-raw-mode-on set-raw-mode-on!
    progn! with-raw-mode
    ;; Trust tag manipulation
    defttag 
    ;; Untouchable manipulation
    remove-untouchable push-untouchable
    ;; File system operations (use tools instead)
    open-input-channel open-output-channel
    read-file-into-string
    ;; State mutation that could escape sandbox
    assign f-put-global
    ;; Loading external code
    include-book ld load include-raw
    ;; Package manipulation  
    defpkg in-package))


Summary
Form:  ( DEFCONST *FORBIDDEN-SYMBOLS* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *FORBIDDEN-SYMBOLS*


In [4]:
;; Check if a symbol is forbidden
(defun forbidden-symbol-p (sym)
  (declare (xargs :guard t))
  (and (symbolp sym)
       (member-equal sym *forbidden-symbols*)))


Since FORBIDDEN-SYMBOL-P is non-recursive, its admission is trivial.
We observe that the type of FORBIDDEN-SYMBOL-P is described by the
theorem 
(OR (CONSP (FORBIDDEN-SYMBOL-P SYM)) (EQUAL (FORBIDDEN-SYMBOL-P SYM) NIL)).
We used the :type-prescription rule MEMBER-EQUAL.

Computing the guard conjecture for FORBIDDEN-SYMBOL-P....

The guard conjecture for FORBIDDEN-SYMBOL-P is trivial to prove, given
the :executable-counterpart of TRUE-LISTP.  FORBIDDEN-SYMBOL-P is compliant
with Common Lisp.

Summary
Form:  ( DEFUN FORBIDDEN-SYMBOL-P ...)
Rules: ((:EXECUTABLE-COUNTERPART TRUE-LISTP)
        (:TYPE-PRESCRIPTION MEMBER-EQUAL))
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 FORBIDDEN-SYMBOL-P


In [5]:
;; Check if form contains any forbidden symbols (recursive)
;; Returns t if form is SAFE, nil if it contains forbidden content
(defun code-safe-p (form)
  (declare (xargs :guard t))
  (cond
   ;; Base case: atom
   ((atom form)
    (not (forbidden-symbol-p form)))
   ;; Recursive case: cons
   (t (and (code-safe-p (car form))
           (code-safe-p (cdr form))))))


The admission of CODE-SAFE-P is trivial, using the relation O< (which
is known to be well-founded on the domain recognized by O-P) and the
measure (ACL2-COUNT FORM).  We observe that the type of CODE-SAFE-P
is described by the theorem 
(OR (EQUAL (CODE-SAFE-P FORM) T) (EQUAL (CODE-SAFE-P FORM) NIL)). 

Computing the guard conjecture for CODE-SAFE-P....

The guard conjecture for CODE-SAFE-P is trivial to prove.  CODE-SAFE-P
is compliant with Common Lisp.

Summary
Form:  ( DEFUN CODE-SAFE-P ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 CODE-SAFE-P


In [6]:
;; Validate code string before evaluation
;; Returns (mv safe-p reason)
(defun validate-code-string (code-str)
  (declare (xargs :guard (stringp code-str) :mode :program))
  ;; Simple pattern checks before parsing
  (cond
   ;; Check for common dangerous patterns in string form
   ((search "defttag" code-str)
    (mv nil "Code contains defttag - trust tags not allowed"))
   ((search "sys-call" code-str)
    (mv nil "Code contains sys-call - system calls not allowed"))
   ((search "set-raw-mode" code-str)
    (mv nil "Code contains set-raw-mode - raw mode not allowed"))
   ((search "progn!" code-str)
    (mv nil "Code contains progn! - raw Lisp forms not allowed"))
   ((search "include-book" code-str)
    (mv nil "Code contains include-book - use tools instead"))
   ((search "include-raw" code-str)
    (mv nil "Code contains include-raw - raw files not allowed"))
   (t (mv t nil))))


Summary
Form:  ( DEFUN VALIDATE-CODE-STRING ...)
Rules: NIL
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)
 VALIDATE-CODE-STRING


In [7]:
;; Format the result of acl2s-compute for LLM consumption
(defun format-compute-result (result)
  (declare (xargs :guard t :mode :program))
  (if (and (consp result) (= (len result) 2))
      (let ((err-flag (first result))
            (value (second result)))
        (if err-flag
            ;; Error case
            (concatenate 'string
                         "ERROR: Evaluation failed\n"
                         "The expression could not be evaluated. "
                         "This may be due to a guard violation, "
                         "undefined function, or runtime error.")
          ;; Success case - format value as string
          (let ((val-str (coerce (packn1 (list value)) 'string)))
            (concatenate 'string "RESULT: " val-str))))
    ;; Unexpected format
    "ERROR: Unexpected result format from evaluator"))


Summary
Form:  ( DEFUN FORMAT-COMPUTE-RESULT ...)
Rules: NIL
Time:  0.03 seconds (prove: 0.00, print: 0.00, other: 0.03)
 FORMAT-COMPUTE-RESULT


In [8]:
;; Parse a string into an ACL2 form
;; Returns (mv erp form) where erp is error message or nil
(defun parse-code-string (code-str state)
  (declare (xargs :guard (stringp code-str)
                  :mode :program
                  :stobjs state))
  ;; Use read-from-string via trans-eval
  (mv-let (erp val state)
    (read-acl2-oracle state)  ; Dummy read for state threading
    (declare (ignore erp val))
    ;; We'll use acl2s-compute to do the parsing safely
    (mv nil (list 'quote code-str) state)))


Summary
Form:  ( DEFUN PARSE-CODE-STRING ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 PARSE-CODE-STRING


In [9]:
;; Execute ACL2 code string safely (Program mode - requires state)
;; Returns (mv error-p result-string state)
;;
;; This is the main entry point for the verified agent's code execution tool.
;; It validates the code, executes it using acl2s-compute, and formats results.
(defun safe-eval (code-str state)
  (declare (xargs :guard (stringp code-str)
                  :mode :program
                  :stobjs state))
  (b* (;; Step 1: Validate code string for forbidden patterns
       ((mv safe-p reason) (validate-code-string code-str))
       ((when (not safe-p))
        (mv t (concatenate 'string "SECURITY ERROR: " reason) state)))
    ;; Step 2: Execute via acl2s-compute 
    ;; Note: We need to drop to raw Lisp to call acl2s-compute
    (mv nil 
        (concatenate 'string 
                     "Code passed validation: " code-str
                     "\nNote: Actual execution requires raw Lisp bridge")
        state)))


Summary
Form:  ( DEFUN SAFE-EVAL ...)
Rules: NIL
Time:  0.03 seconds (prove: 0.00, print: 0.00, other: 0.03)
 SAFE-EVAL


In [10]:
;; Tool spec for code execution (uses verified-agent types when loaded together)
;; Note: This is just the constant data; actual tool-spec requires verified-agent
(defconst *code-exec-tool-spec*
  '(:name execute-acl2-code
    :required-access 0      ; No file access needed
    :requires-execute t     ; REQUIRES execute permission!
    :token-cost 500         ; Generous token budget for results
    :time-cost 30))         ; 30 seconds max


Summary
Form:  ( DEFCONST *CODE-EXEC-TOOL-SPEC* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *CODE-EXEC-TOOL-SPEC*


In [11]:
;; Load the raw Lisp implementation
;; This provides: execute-acl2-code, execute-acl2-query, execute-acl2-event
(defttag :code-exec)
(include-raw "code-exec-raw.lsp")


TTAG NOTE: Adding ttag :CODE-EXEC from the top level loop.
 :CODE-EXEC


ACL2 !>>(PROGN!
         (SET-RAW-MODE T)
         (DEFUN RAW-COMPILE (NAME ERROR-ON-FAIL ON-FAIL STATE)
          (COND
           ((NOT (MEMBER :CLTL2 *FEATURES*))
            (COMPILE-FILE (EXTEND-PATHNAME (CBD) NAME STATE)))
           (T
            (HANDLER-CASE
             (COMPILE-FILE (EXTEND-PATHNAME (CBD) NAME STATE))
             (ERROR
              (CONDITION)
              (IF ERROR-ON-FAIL
               (LET ((CONDITION-STR (FORMAT NIL "~a" CONDITION)))
                (ER
                 HARD 'INCLUDE-RAW
                 "Compilation of ~x0 failed with the following message:~%~@1~%"
                 NAME CONDITION-STR))
               (EVAL
                (CONS
                  'LET
                  (CONS (CONS (CONS 'CONDITION
                                    (CONS (CONS 'QUOTE (CONS CONDITION 'NIL))
                                          'NIL))
                              'NIL)
 

In [ ]:
#||
Example usage in ACL2 (after including this book):

In [ ]:
;; Method 1: From raw Lisp
:q
(execute-acl2-code "(+ 1 2)")
; => "RESULT: 3"
(lp)

In [14]:
;; Method 2: Using make-event to capture result into a constant
(make-event
 (let ((result (execute-acl2-code "(append '(1 2) '(3 4))")))
   (mv nil `(defconst *last-result* ,result) state)))



ACL2 Error [Translate] in ( MAKE-EVENT (LET ...)):  The symbol 
EXECUTE-ACL2-CODE (in package "ACL2") has neither a function nor macro
definition in ACL2.  Please define it.  See :DOC near-misses.  Note:
this error occurred in the context 
(EXECUTE-ACL2-CODE "(append '(1 2) '(3 4))").


Summary
Form:  ( MAKE-EVENT (LET ...))
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)

ACL2 Error [Failure] in ( MAKE-EVENT (LET ...)):  See :DOC failure.

******** FAILED ********


In [ ]:
;; Method 3: For queries (thm, test?)
:q
(execute-acl2-query "(thm (equal (+ 1 1) 2))")
; => "QUERY SUCCEEDED..."
(lp)
||#